In [ ]:
import os
import geopandas as gpd
from shapely.geometry import Point, MultiLineString
from shapely.ops import transform, unary_union
from functools import partial
import pyproj
import pandas as pd

def calculate_distance(geometry):
    if geometry.is_empty:
        return 0
    else:
        # Project the geometry to a CRS with units in meters
        project = partial(
            pyproj.transform,
            pyproj.Proj(init='epsg:4326'),  # WGS84 coordinate reference system
            pyproj.Proj(init='epsg:3857')   # Web Mercator projection with meters as units
        )
        projected_geometry = transform(project, geometry)
        return projected_geometry.length

def create_intersection_nodes(shapefile_path):
    # Read the shapefile using geopandas
    gdf = gpd.read_file(shapefile_path)

    # Combine all line geometries into a single MultiLineString
    all_lines = unary_union(gdf['geometry'])

    # Create nodes only at intersections
    nodes = []
    for line in all_lines.geoms:
        if line.geom_type == 'LineString':
            intersection_points = gdf['geometry'].apply(lambda x: line.intersection(x)).explode(index_parts=False)
            point_nodes = [Point(xy) for x in intersection_points if not x.is_empty and x.geom_type == 'Point' and (xy := x.coords[0])]
            nodes.extend(point_nodes)

    # Create a GeoDataFrame for nodes
    nodes_gdf = gpd.GeoDataFrame(geometry=nodes, crs='EPSG:3857')

    return nodes_gdf

def save_nodes_to_files(nodes_gdf, output_folder):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Save nodes to CSV file
    csv_path = os.path.join(output_folder, 'output_nodes.csv')
    nodes_gdf.to_csv(csv_path, index=False)
    print(f'Nodes CSV file saved to {csv_path}')

    # Save nodes to Shapefile
    shp_path = os.path.join(output_folder, 'output_nodes.shp')
    nodes_gdf.to_file(shp_path)
    print(f'Nodes Shapefile saved to {shp_path}')

if __name__ == "__main__":
    # Replace 'your_shapefile.shp' with the actual path to your shapefile
    shapefile_path = r"filepath"
    # Replace 'your_folder_name' with the desired folder name
    output_folder = 'IntersectionNodesOutput'

    try:
        nodes_gdf = create_intersection_nodes(shapefile_path)
        save_nodes_to_files(nodes_gdf, output_folder)
    except Exception as e:
        print(f'Error: {e}')
